In [1]:
from datasets import load_dataset 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, GPTQConfig
import torch
import time
import pandas as pd
dataset = load_dataset("jigsaw_toxicity_pred", data_dir="/workspaces/LLM-Experimentation-Capstone/00_source_data/jigsaw_toxicity")
dataset = dataset['train'][0:10_000]
texts = dataset['comment_text']

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("andreas122001/bloomz-3b-wiki-detector")
gptq_config = GPTQConfig(bits=4, dataset = "c4", tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("andreas122001/bloomz-3b-wiki-detector", quantization_config=gptq_config)


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.31s/it]
Repo card metadata block was not found. Setting CardData to empty.
Quantizing transformer.h blocks :   7%|▋         | 2/30 [00:53<12:18, 26.37s/it]

In [ ]:
tokenized_texts = [tokenizer(x, padding=True, truncation=True, return_tensors="pt", max_length=512) for x in texts]
time_taken = []
for text in tokenized_texts:
    start_time = time.time()
    with torch.no_grad():
        logits = model(**text).logits
    end_time = time.time()
    time_taken.append(end_time - start_time)
output = pd.DataFrame({"bert_base":time_taken})
output.to_csv("bert_base_inference.csv", index=False)